<a href="https://colab.research.google.com/github/zakrzewki/pracaMgr/blob/master/Mgr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First of all, take fresh data from repo

In [0]:
!git clone https://github.com/zakrzewki/pracaMgr

Cloning into 'pracaMgr'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 11390 (delta 34), reused 40 (delta 17), pack-reused 11332
Receiving objects: 100% (11390/11390), 93.98 MiB | 7.54 MiB/s, done.
Resolving deltas: 100% (3072/3072), done.
Checking out files: 100% (8833/8833), done.


**Connect to Google Drive**

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Fast checker if connection and exporting files to Google Drive are working correctly

In [0]:
!python3 /content/pracaMgr/GoogleDriveChecker.py

Following path exists, file was successfully exported


**Downgrade keras** (`train_frcnn.py` throws error when it's 2.2.4 version)

In [0]:
!pip install keras==2.1.6

     |████████████████████████████████| 348kB 34.5MB/s 
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


**Create a connection with Kaggle DB for downloading data**

In [0]:
!mkdir /content/.kaggle

mkdir: cannot create directory ‘/content/.kaggle’: File exists


In [0]:
import json

token = {"username":"michzak","key":"558c3de4322bd2128047049c6ea44e42"}

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

Add folders necessary for dividing test and train data

In [0]:
!mkdir /content/pracaMgr/input/

In [0]:
!mkdir /content/pracaMgr/input/test_v2

In [0]:
!mkdir /content/pracaMgr/input/train_v2

Download data from Kaggle using following commands:

In [0]:
!kaggle competitions download -c airbus-ship-detection -p /content/pracaMgr/input

  0% 0.00/274k [00:00<?, ?B/s]
100% 274k/274k [00:00<00:00, 40.7MB/s]
 50% 9.00M/18.0M [00:01<00:01, 8.51MB/s]
100% 18.0M/18.0M [00:01<00:00, 16.9MB/s]
 99% 2.11G/2.12G [00:47<00:00, 21.7MB/s]
100% 2.12G/2.12G [00:47<00:00, 47.5MB/s]
100% 26.4G/26.4G [08:31<00:00, 70.2MB/s]
100% 26.4G/26.4G [08:31<00:00, 55.4MB/s]


Unpack all data to selected folder

In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/pracaMgr/input/train_ship_segmentations_v2.csv.zip", 'r')
zip_ref.extractall("/content/pracaMgr/input/")
zip_ref.close()

In [0]:
zip_ref = zipfile.ZipFile("/content/pracaMgr/input/test_v2.zip", 'r')
zip_ref.extractall("/content/pracaMgr/input/test_v2")
zip_ref.close()

In [0]:
zip_ref = zipfile.ZipFile("/content/pracaMgr/input/train_v2.zip", 'r')
zip_ref.extractall("/content/pracaMgr/input/train_v2")
zip_ref.close()

Move unnecessary zip folders to somewhere else

In [0]:
!mkdir /kaggle_zips

In [0]:
import os

os.rename("/content/pracaMgr/input/train_ship_segmentations_v2.csv.zip", "/kaggle_zips/train_ship_segmentations_v2.csv.zip")

In [0]:
os.rename("/content/pracaMgr/input/train_v2.zip", "/kaggle_zips/train_v2.zip")

In [0]:
os.rename("/content/pracaMgr/input/test_v2.zip", "/kaggle_zips/test_v2.zip")

Using RLE algorithm take some example data for creating Bounding Boxes for training purposes

In [0]:
!python3 /content/pracaMgr/Decoder.py

Number of ships:  81723
Going with full check
Checked ships: 81723
Incorrect ships: 1371
Correct ships: 80352
Incorrect to correct ratio: 0.01706242532855436
Finished


In [0]:
!python3 /content/pracaMgr/Background_decoder.py

Number of images without ships: 150000
Going with full check with 150000 images
Checked images: 150000
Finished


**Move it to the correct folder**



In [0]:
if os.path.exists("/content/entry_data.csv"):
  os.rename("/content/entry_data.csv", "/content/pracaMgr/entry_data.csv")

**Train stuff!**

In [0]:
!python3 /content/pracaMgr/train_frcnn.py simple -p /content/pracaMgr/entry_data.csv --num_epochs=40

Using TensorFlow backend.
Parsing annotation files
Found class name with special name bg. Will be treated as a background region (this is usually for hard negative mining).
Premature end of JPEG file
Training images per class:
{'bg': 150000, 'ship': 80352}
Num classes (including bg) = 2
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 160081
Num val samples 0
Num test samples 31893
W0707 10:55:10.518693 139908715136896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 10:55:10.530035 139908715136896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:508: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 10:55:10.554330 139908715136896 deprecation_wrapper.py:119

**For testing purposes, take the best weights from Google Drive**

In [0]:
import shutil
shutil.copy("/content/drive/My Drive/pracaMgr/Weights/Best/1699_dp.hdf5", "/content/weights.hdf5")

'/content/weights.hdf5'

Pull any changes new changes, if it's necessary

In [0]:
%cd /content/pracaMgr
!git pull

/content/pracaMgr
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/zakrzewki/pracaMgr
   868f7f9..77af64f  master     -> origin/master
Updating 868f7f9..77af64f
Fast-forward
 test_frcnn.py | 6 +++---
 1 file changed, 3 insertions(+), 3 deletions(-)


**Test it, how it's working**

In [0]:
!python3 /content/pracaMgr/test_frcnn.py --p /content/pracaMgr/input/test_v2/ --input_weight_path /content/weights.hdf5 --config_filename /content/config.pickle

**Export results to Google Drive**

In [0]:
shutil.move("/content/results.csv", "content/gdrive/My Drive/pracaMgr/Weights/results.csv")
shutil.move("/content/ship_detected.csv", "content/gdrive/My Drive/pracaMgr/Weights/ship_detected.csv")